In [1]:
import numpy as np
from typing import List
import logging
import os
import sys
from scipy import stats
#配置输出日志
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s]:%(message)s"
)
SOURCE_DATA="data_ps3_ex2.csv"

In [2]:
#读取csv数据
def load_data(src:str,ingnore_head:bool=True,column_select:List[int]=[],encoding="utf-8",sep=",")->np.ndarray:
    '''
    Args:
        src:file path
        ignore_head:whether ignore the first row
        columns_select:list of int,which you want to select
        encoding:file encoding style,default is utf-8
        sep:delimiter,default is ','
    Return:
        data_array:np.ndarray object of int32 dtype

    Raise:
        FileNotFoundError:if src is not exist!
    '''
    if not os.path.exists(src):
        logging.error("%s is not found!"%src)
        sys.exit(0)

    with open(src,mode="r",encoding=encoding) as f:
        if ingnore_head:
            header_names=f.readline().strip().split(sep)
            logging.info("we ignore first head....%s"%str(header_names))
        data_list=[]
        for index,line in enumerate(f,start=1):
            split_line=line.strip().split(sep)
            if column_select:
                item_select=[split_line[item] for item in column_select]
                data_list.append(item_select)
            else:
                data_list.append(split_line)
            if index%20==0:
                logging.info("Reading rows for %d"%index)
    data_array=np.array(
        data_list,
        dtype=np.float32
    )
    return data_array


'''
param theta as follows:
-----------------------
    miu
    theta_x
    theta_y
    though_xy
-----------------------
so p=4,respect that number of params is four

equation is like that:
---------------------------------------------
    x-miu
    y-miu
    (x-miu)**2-theta_x**2
    (y_miu)**2-theta_y**2
    (x-miu)(y-miu)-theta_x*theta_y*though_xy
so q=5,respect that number of equations is five
-----------------------------------------------
'''

#获取统计量,直接推倒公式得出
def get_stat_values(data:np.ndarray):
    """
    获取一些统计值,会在G(theta)中用到,包括x,y的均值,平方和,乘积和,和
    """
    x,y=data[0,],data[1,]
    print(np.std(x),np.std(y))
    x_mean_square=np.mean(
        np.square(x)
    )
    x_mean=np.mean(x)

    y_mean_square=np.mean(
        np.square(y)
    )
    y_mean=np.mean(y)

    xy_mean_multi=np.mean(x*y)
    xy_mean_sum=np.mean(x+y)

    print_info='''stat info of variable x,y
    ------------------------------------------------
    x_mean_square:%.2f
    x_mean:%.2f

    y_mean_square:%.2f
    y_mean:%.2f

    xy_mean_multi:%.2f
    xy_mean_sum:%.2f
    ------------------------------------------------
    '''%(x_mean_square,x_mean,y_mean_square,y_mean,xy_mean_multi,xy_mean_sum)
    logging.info(print_info)
    return [
        x_mean_square,x_mean,
        y_mean_square,y_mean,
        xy_mean_multi,xy_mean_sum
    ]


# data_array=load_data(
#     src=SOURCE_DATA,
#     column_select=[1,2]
# )

# get_stat_values(
#     data=data_array
# )
# print(data_array)

data=np.random.randn(1000,2)

In [3]:
#定义计算过程
def _worker(params,stat_data,W=None):
    """
    Args:
        params:mu,sigma_x,sigma_y,through_xy，给定的theta初始解，用来梯度下降，优化Q value
        stat_data:在计算G(theta)向量时用到的常量,可直接推导公式
        W:方程加权矩阵，qxq矩阵，q是你的方程个数，如果为None,就用单位矩阵替代，用在gmm的第一步估计
    Returns:
        params:通过梯度下降找到的比较优的解
    """
    x_mean_square,x_mean,y_mean_square,y_mean,xy_mean_multi,xy_mean_sum=stat_data
    mu,sigma_x,sigma_y,though_xy=params[0],params[1],params[2],params[3]
    G_param=np.array(
        [
            x_mean-mu,
            y_mean-mu,
            x_mean_square-2*x_mean*mu+mu**2-sigma_x**2,
            y_mean_square-2*y_mean*mu+mu**2-sigma_y**2,
            xy_mean_multi-xy_mean_sum*mu+mu**2-though_xy*sigma_x*sigma_y
        ]
    ).reshape(-1,1)

    G_param_T=np.transpose(G_param)

    #方程的个数
    q_number=G_param.shape[0]
    #参数的个数
    p_number=len(params)

    #定义梯度矩阵,参数个数=4,方程个数=5 so the shape is (5,4)
    G_grad=np.array(
        [
            [-1,0,0,0],
            [-1,0,0,0],
            [2*(mu-x_mean),-2*sigma_x,0,0],
            [2*(mu-y_mean),0,-2*sigma_y,0],
            [2*(mu-xy_mean_sum),-sigma_x*though_xy,-sigma_y*though_xy,-sigma_x*sigma_y]
        ]
    )

    #获取其转置矩阵
    G_grad_T=np.transpose(G_grad)
    
    if W is None:
    #使用W为初始单位方阵
        W=np.eye(q_number,dtype=np.float32)

    #计算Q值
    Q_params=np.dot(
        np.dot(G_param_T,W),G_param
    )
    
    #计算Q关于 参数的梯度
    Q_grad=2*np.dot(
        np.dot(G_grad_T,W),G_param
    )

    return Q_params,Q_grad


In [4]:
def gmm_first_step(lr=0.01,steps=20,verbose=True):
    """
    lr:梯度下降的学习率，就是步长
    verbose:是否显示迭代过程,默认显示
    """
    # data=load_data(
    #     src=SOURCE_DATA,
    #     column_select=[1,2]
    # )

    # x_mean_square,x_mean,y_mean_square,y_mean,xy_mean_multi,xy_mean_sum=get_stat_values(data)
    stat_data=get_stat_values(data)
    params={
        "mu":0.1,
        "sigma_x":1.2,
        "sigma_y":1.5,
        "though_xy":0
    }

    mu=params["mu"]
    sigma_x=params["sigma_x"]
    sigma_y=params["sigma_y"]
    though_xy=params["though_xy"]

    init_params=np.array([mu,sigma_x,sigma_y,though_xy])
    #定义G矩阵
    _params=init_params
    
    logging.info("beging evalute theta by frist GMM FIRST STEP....")
    for step in range(steps):
        Q_params,Q_grad=_worker(_params,stat_data)
        logging.info("iter at %d,the Q value is %.5f"%(step,Q_params[0,0]))
        _params=_params-lr*Q_grad.reshape(-1,)
        # print(_params)
    return _params


In [5]:
def evaliuate_W_by_first_step(_params):
    """
    根据 gmm first step的结果计算W
    Args:   
        _param:gmm first求的theta
    Return:
        W:加权矩阵
    """
    #unwarp param
    mu,sigma_x,sigma_y,though_xy=_params
    def _calc(x,y):
        g_param=np.array(
                [
                    x-mu,
                    y-mu,
                    (x-mu)**2-sigma_x**2,
                    (y-mu)**2-sigma_y**2,
                    (x-mu)*(y-mu)-sigma_x*sigma_y*though_xy
                ]
            ).reshape(-1,1)
        g_param_T=np.transpose(g_param)
        result=np.dot(g_param,g_param_T)
        return result
    
    rows,_=data.shape
    W=np.zeros(shape=(5,5))
    #严格按照公式，先求矩阵乘积
    for i in range(rows):
        x,y=data[i,]
        W=W+_calc(x, y)

    #再求其逆矩阵
    W=np.linalg.inv(W)
    return W

In [6]:
def gmm_second_step(params,W,lr=0.01,steps=20,verbose=True):
    """
    根据上一步求的W求解theta
    参数含义同上
    """
    logging.info("begin evalue theta by GMM SECOND STEP...")
    stat_data=get_stat_values(data)
    lr=0.1
    for step in range(steps):
        Q_params,Q_grad=_worker(params,stat_data,W=W) #指定第一次估计的W
        logging.info("iter at %d,the Q value is %.5f"%(step,Q_params[0,0]))
        params=params-lr*Q_grad.reshape(-1,)
    return params

In [7]:
def gmm_theta_var(params,W):
    """
    求解theta 的方差，严格按照公式
    Args:
        params:theta
        W:加权矩阵,B的逆矩阵就是W
    """
    x_mean_square,x_mean,y_mean_square,y_mean,xy_mean_multi,xy_mean_sum=get_stat_values(data)
    mu,sigma_x,sigma_y,though_xy=params[0],params[1],params[2],params[3]
    G_param=np.array(
        [
            x_mean-mu,
            y_mean-mu,
            x_mean_square-2*x_mean*mu+mu**2-sigma_x**2,
            y_mean_square-2*y_mean*mu+mu**2-sigma_y**2,
            xy_mean_multi-xy_mean_sum*mu+mu**2-though_xy*sigma_x*sigma_y
        ]
    ).reshape(-1,1)

    G_grad=np.array(
        [
            [-1,0,0,0],
            [-1,0,0,0],
            [2*(mu-x_mean),-2*sigma_x,0,0],
            [2*(mu-y_mean),0,-2*sigma_y,0],
            [2*(mu-xy_mean_sum),-sigma_x*though_xy,-sigma_y*though_xy,sigma_x*sigma_y]
        ]
    )

    G_grad_transpose=np.transpose(G_grad)
    #np.dot 这个api是求解矩阵叉乘
    theta_variance=np.mean(
        np.dot(
            np.dot(G_grad_transpose,W),G_grad
        )
    )
    return theta_variance

In [8]:
def J_test(W,theta):
    """
    我不晓得公式
    """
    rows,_=data.shape
    mu,sigma_x,sigma_y,though_xy=theta
    x_mean_square,x_mean,y_mean_square,y_mean,xy_mean_multi,xy_mean_sum=get_stat_values(data)
    G_param=np.array(
        [
            x_mean-mu,
            y_mean-mu,
            x_mean_square-2*x_mean*mu+mu**2-sigma_x**2,
            y_mean_square-2*y_mean*mu+mu**2-sigma_y**2,
            xy_mean_multi-xy_mean_sum*mu+mu**2-though_xy*sigma_x*sigma_y
        ]
    ).reshape(-1,1)
    G_param_T=np.transpose(G_param)
    J=rows*np.dot(
        np.dot(G_param_T,W),G_param
    )
    p=stats.chi2.cdf(J,df=4)
    return J,p

#### G_theta 对theta的导数  theta=\begin{matrix} \mu&\sigma_x&\sigma_y&\rho_xy\end{matrix}

$${\left[ {\begin{matrix}
-1&0&0&0\\
-1&0&0&0\\
2\times(\mu-x)&-2\times\sigma_x&0&0\\
2\times(\mu-y)&0&2\times\sigma_y&0\\
2\times(\mu-(x+y))&-\sigma_y\times\rho_xy&-\sigma_x\times\rho_xy&-\sigma_x\times\sigma_y
\end{matrix}} \right]}$$

#### gmm first step

In [9]:
first_step_params=gmm_first_step()
print("theta is :\n",first_step_params)

2021-05-13 07:46:22,837 [INFO]:stat info of variable x,y
    ------------------------------------------------
    x_mean_square:0.20
    x_mean:0.43

    y_mean_square:0.31
    y_mean:-0.47

    xy_mean_multi:-0.17
    xy_mean_sum:-0.04
    ------------------------------------------------
    
2021-05-13 07:46:22,838 [INFO]:beging evalute theta by frist GMM FIRST STEP....
2021-05-13 07:46:22,841 [INFO]:iter at 0,the Q value is 5.58302
2021-05-13 07:46:22,843 [INFO]:iter at 1,the Q value is 4.10913
2021-05-13 07:46:22,846 [INFO]:iter at 2,the Q value is 3.19927
2021-05-13 07:46:22,848 [INFO]:iter at 3,the Q value is 2.59491
2021-05-13 07:46:22,850 [INFO]:iter at 4,the Q value is 2.17221
2021-05-13 07:46:22,851 [INFO]:iter at 5,the Q value is 1.86479
2021-05-13 07:46:22,852 [INFO]:iter at 6,the Q value is 1.63417
2021-05-13 07:46:22,854 [INFO]:iter at 7,the Q value is 1.45669
2021-05-13 07:46:22,855 [INFO]:iter at 8,the Q value is 1.31707
2021-05-13 07:46:22,857 [INFO]:iter at 9,the Q va

0.11118444858709076 0.28506283032469
theta is :
 [ 0.15682362  0.67996357  0.8631801  -0.04676108]


#### 根据gmm first step的\theta 计算加权矩阵W

In [10]:
W=evaliuate_W_by_first_step(first_step_params)
print("方程加权矩阵:\n",W)

方程加权矩阵:
 [[ 1.08720401e-03 -2.01354110e-06  1.53321100e-04 -7.94013858e-06
   1.15917568e-04]
 [-2.01354110e-06  9.96727211e-04  1.16379984e-06  1.66666704e-04
   1.12607306e-04]
 [ 1.53321100e-04  1.16379984e-06  4.67174371e-04 -3.61200284e-05
  -1.94869848e-05]
 [-7.94013858e-06  1.66666704e-04 -3.61200284e-05  3.99196848e-04
   4.31188472e-05]
 [ 1.15917568e-04  1.12607306e-04 -1.94869848e-05  4.31188472e-05
   9.37409672e-04]]


#### 根据gmm 上一步的W第二次计算 \theta 参数

In [11]:
second_params=gmm_second_step(
    params=first_step_params,
    W=W
)
print("theta second params is :\n",second_params)

2021-05-13 07:46:22,916 [INFO]:begin evalue theta by GMM SECOND STEP...
2021-05-13 07:46:22,918 [INFO]:stat info of variable x,y
    ------------------------------------------------
    x_mean_square:0.20
    x_mean:0.43

    y_mean_square:0.31
    y_mean:-0.47

    xy_mean_multi:-0.17
    xy_mean_sum:-0.04
    ------------------------------------------------
    
2021-05-13 07:46:22,920 [INFO]:iter at 0,the Q value is 0.00061
2021-05-13 07:46:22,922 [INFO]:iter at 1,the Q value is 0.00061
2021-05-13 07:46:22,923 [INFO]:iter at 2,the Q value is 0.00061
2021-05-13 07:46:22,925 [INFO]:iter at 3,the Q value is 0.00061
2021-05-13 07:46:22,927 [INFO]:iter at 4,the Q value is 0.00061
2021-05-13 07:46:22,929 [INFO]:iter at 5,the Q value is 0.00061
2021-05-13 07:46:22,930 [INFO]:iter at 6,the Q value is 0.00061
2021-05-13 07:46:22,933 [INFO]:iter at 7,the Q value is 0.00061
2021-05-13 07:46:22,934 [INFO]:iter at 8,the Q value is 0.00061
2021-05-13 07:46:22,936 [INFO]:iter at 9,the Q value is 0

0.11118444858709076 0.28506283032469
theta second params is :
 [ 0.15601972  0.67932849  0.86179113 -0.04711606]


#### 计算theta的方差

In [12]:
theta_variance=gmm_theta_var(second_params, W)

# print(second_params)
print("Theta VARIANCE is %.6f"%theta_variance)

2021-05-13 07:46:22,963 [INFO]:stat info of variable x,y
    ------------------------------------------------
    x_mean_square:0.20
    x_mean:0.43

    y_mean_square:0.31
    y_mean:-0.47

    xy_mean_multi:-0.17
    xy_mean_sum:-0.04
    ------------------------------------------------
    


0.11118444858709076 0.28506283032469
Theta VARIANCE is 0.000321


#### 计算J统计量

In [13]:
J,p=J_test(W,second_params)
print("J_stat\t p\t")
print("%.4f\t %.4f\t"%(J,p))

2021-05-13 07:46:22,974 [INFO]:stat info of variable x,y
    ------------------------------------------------
    x_mean_square:0.20
    x_mean:0.43

    y_mean_square:0.31
    y_mean:-0.47

    xy_mean_multi:-0.17
    xy_mean_sum:-0.04
    ------------------------------------------------
    


0.11118444858709076 0.28506283032469
J_stat	 p	
0.6134	 0.0384	
